In [1]:
import pandas as pd

In [2]:
# Read data
file_name = "train.csv"
train = pd.read_csv(file_name)


In [3]:
# Explore the dataset
print train.shape
train.head(5)
train.describe()

(891, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
# Check missing values in the dataset
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# Explor Name

def title_checker (name):
    title_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Dr.', 'Rev.']
    split_name = name.split(' ')
    for word in split_name:
        if word in title_list:
            index = title_list.index(word)
            return title_list[index]
       
        
train['Title'] = train['Name'].apply(title_checker)
train.head()
train['Title'].isnull().sum()

#title_checker('Braund, Mr. Owen Harris')

14

In [6]:
# fill the missing values in "Age" column with the mean

print 'Mean of age:', train['Age'].mean()
print 'Mode of age:', train['Age'].mode()[0]
print 'Std of age:', train['Age'].std()
sigma = train['Age'].std()
mu = train['Age'].mean()



print train['Age'].describe()

# Randomly generate age with standard diviation 
import random
nums = [random.gauss(mu, sigma) for _ in range(10)]
random.gauss(mu, sigma) 
#abs(nums)

Mean of age: 29.6991176471
Mode of age: 24.0
Std of age: 14.5264973323
count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64


33.57804823458039

In [7]:
# Feature Enginerring
def cabin_categorizer (cabin):
    cabin = str(cabin)
    return cabin[0]

def impute_cabin_with_class(pclass):
    if pclass == 1:
        return 'C'
    elif pclass == 2:
        return 'E'
    elif pclass == 3:
        return 'F'
           
def featuring_process(df):
    # add family size column
    df['Family_size'] = 1 + df['Parch'] + df['SibSp']
    
    # add cabin_cat by Cabin number
    df['Cabin_cat'] = df['Cabin'].apply(cabin_categorizer)
    missing_cabin_index = df[df['Cabin_cat'] == 'n'].index
    for index in missing_cabin_index:
        pclass = df.loc[index, 'Pclass']
        df.loc[index, 'Cabin_cat'] = impute_cabin_with_class(pclass)
        
    # add Title by Name
    df['Title'] = df['Name'].apply(title_checker)
    df['Title'].fillna('no_title', inplace = True)

featuring_process(train)

train.isnull().sum()




PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Title            0
Family_size      0
Cabin_cat        0
dtype: int64

In [68]:
# Convert feature_list into numeric

def assign_into_object(df, feature_list):
    type_list = df.dtypes
    for col in feature_list:
        if type_list[col] == 'object':
            unique_values, result_dict = transfor_to_numeric(df, col)
            assign_numeric_values(df, col, unique_values, result_dict)
    return df

def transfor_to_numeric (df, col_name):
    values = df[col_name]
    unique_values = set(values)
    unique_values = list(unique_values)
    len_of_values = len(unique_values)
    result_dict = dict( zip(unique_values, range(len_of_values)))
    return unique_values, result_dict

def assign_numeric_values(df, col_name, unique_values, result_dict):
    for value in unique_values:
        df[col_name][df[col_name] == value ] = result_dict[value]
    return df


In [71]:
# Fill the missing ages with randomly assigned values
#train['Age'].fillna(random.gauss(mu, sigma), inplace = True)

# Exclude all missing ages
#train.dropna(inplace = True, subset = ['Age'])


# Method3: Build a prediction model for missing ages
def missing_age_predictor(df):
    featuring_process(df)
    age_feature_list = ['Fare', 'SibSp','Pclass', 'Cabin_cat','Parch']
    assign_into_object(df, age_feature_list)
    
    age_df = df[['Age','Fare', 'SibSp','Pclass', 'Cabin_cat','Parch']]
    knownAge = df.loc[(df.Age.notnull())]
    unknownAge = df.loc[(df.Age.isnull())]
    
    age_target =knownAge['Age'].values
    age_features = knownAge[age_feature_list].values

    from sklearn import linear_model
    regr = linear_model.LinearRegression()
    regr.fit(age_features, age_target)
    predictAges = regr.predict(unknownAge[age_feature_list].values)
    df.loc[(df.Age.isnull()), 'Age'] = predictAges
    
    return df

age_feature_list = ['Fare', 'SibSp','Pclass', 'Cabin_cat','Parch', 'Title']
assign_into_object(train, age_feature_list)

age_df = train[['Age','Fare', 'SibSp','Pclass', 'Cabin_cat','Parch', 'Title']]
knownAge = train.loc[(train.Age.notnull())]
unknownAge = train.loc[(train.Age.isnull())]
print len(knownAge)

age_label = knownAge['Age'].values
age_features = knownAge[age_feature_list].values


# Split to train and test dataset
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels,test_labels = train_test_split(
age_features, age_label, test_size = 0.2, random_state = 0 )

print train_features.shape
print test_features.shape
print train_labels.shape
print test_labels.shape





/Users/ruyi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


714
(571, 6)
(143, 6)
(571,)
(143,)


In [70]:
# Age prediction - SVR

from sklearn.svm import SVR
svr_lin = SVR(kernel='linear')
svr_lin.fit(train_features, train_labels)
pred = svr_lin.predict(test_features)

from sklearn.metrics import explained_variance_score
print explained_variance_score(test_labels, pred)


first_score = 0.256824329357
second_score = 0.263137089874 #with title

0.263137089874


In [72]:
# Age prediction - Linear_model.Lasso

from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg.fit(train_features, train_labels)
pred = reg.predict(test_features)
print 'prediction', pred.shape
print 'test_labels:', test_labels.shape
#reg.score(test_features, test_labels)

from sklearn.metrics import explained_variance_score
print explained_variance_score(test_labels, pred)

first_score = 0.259438390473

prediction (143,)
test_labels: (143,)
0.266714714381


In [ ]:

train = missing_age_predictor(train)
print train.isnull().sum()
train.shape

train.head()

In [444]:
# Exploring Cabin data

cabin_df = train[['Cabin', 'Pclass']]
cabin_df.dropna(inplace = True)
print cabin_df.head(3)
print cabin_df.shape





  Cabin  Pclass
1   C85       1
3  C123       1
6   E46       1
(204, 2)


/Users/ruyi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [533]:
featuring_process(train)
print train.shape
train.isnull().sum()

(891, 14)


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Family_size      0
Cabin_cat        0
dtype: int64

In [380]:
set(train['Cabin_cat'])

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [509]:
train.shape
y2 = ["A","1","4","F","A","1","4","F"]
S = set(y2)
print S
print len(S)
print range(3)
print zip(S, range(len(S)))
D = dict( zip(S, range(len(S))) )
print D

print train.columns.values



unique_values, result_dict = transfor_to_numeric(train, "Sex")
print 'zip:', unique_values, result_dict


    
assign_numeric_values (train, "Sex", unique_values, result_dict)
train.head()


set(['A', '1', '4', 'F'])
4
[0, 1, 2]
[('A', 0), ('1', 1), ('4', 2), ('F', 3)]
{'A': 0, '1': 1, '4': 2, 'F': 3}
['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked' 'Family_size' 'Cabin_cat']
zip: ['male', 'female'] {'male': 0, 'female': 1}


/Users/ruyi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_size,Cabin_cat
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,2,F
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,2,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,F
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,2,C
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,1,F


In [75]:

feature_list = ['Sex', 'Age', 'Family_size', 'Cabin_cat', 'Title']
train['Age'].fillna(random.gauss(mu, sigma), inplace = True)

assign_into_object(train, feature_list)
train.isnull().sum()

/Users/ruyi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Family_size      0
Cabin_cat        0
Title            0
dtype: int64

In [405]:
# exclude datapoint without age number

#train["Age"][train["Age"].isnull()] = 9999
#print set(train["Age"])


set([0, 1, 2, 3, 4, 5, 6, 7])


In [76]:
# Prepare for estimator testing
# Whole dataset
labels = train['Survived'].values
features = train[feature_list].values



In [ ]:

# Split to train and test dataset
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels,test_labels = train_test_split(
features, labels, test_size = 0.15, random_state = 0 )


print "Train features:", train_features.shape
print "Train labels:", train_labels.shape
print "Test features:", test_features.shape
print "Test labels:",test_labels.shape

In [357]:
# Use KFold to split train/test data

from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression


# Load the data:
#iris = load_iris()
#X, y = iris.data, iris.target
 
# Split in 5 folds:
#n = len(X)
#kf = KFold(n_splits=5)
#fold = 0
#for train_idx, test_idx in kf.split(X):
#    fold += 1
#    X_train, X_test = X[train_idx], X[test_idx]
#    y_train, y_test = y[train_idx], y[test_idx]
#    clf = LogisticRegression().fit(X_train, y_train)
#    score = clf.score(X_test, y_test)
#    print("Score for fold %d: %.3f" % (fold, score))





# Split in 5 folds
n = len(train)
kf = KFold(n_splits=10)
fold = 0
for train_idx, test_idx in kf.split(features):
    fold += 1
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]
    clf = LogisticRegression().fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("Score for fold %d: %.3f" % (fold, score))

Score for fold 1: 0.800
Score for fold 2: 0.820
Score for fold 3: 0.775
Score for fold 4: 0.809
Score for fold 5: 0.764
Score for fold 6: 0.764
Score for fold 7: 0.764
Score for fold 8: 0.753
Score for fold 9: 0.865
Score for fold 10: 0.775


In [549]:
# LinearSVC
import time
from sklearn import svm

print 'clf name: LinearSVC'
clf = svm.LinearSVC()
'''
t0 = time.time()
clf.fit(train_features, train_labels)
print "training time:", round(time.time()-t0, 3), "s"

t0 = time.time()
pred = clf.predict(test_features)
print "predicting time:", round(time.time()-t0, 3), "s"

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred, test_labels)
print 'Accuracy score:', accuracy
'''

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(clf, features, labels, cv=5)
print 'Cross-validated scores:', scores.mean()


clf name: LinearSVC
Cross-validated scores: 0.751946122172


In [77]:
# naive_bayes_clf
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
'''   
print 'clf name: Naive Bayes'
t0 = time.time()
clf.fit(train_features, train_labels)
print "training time:", round(time.time()-t0, 3), "s"

t0 = time.time()
pred = clf.predict(test_features)
print "predicting time:", round(time.time()-t0, 3), "s"

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred, test_labels)
print 'accuracy score:', accuracy

print 'GaussianNB priors:', clf.class_prior_
'''

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(clf, features, labels, cv=5)
print 'Cross-validated scores:', scores.mean()

clf.fit(features, labels)

previous_score = 0.791285015499
seconde_score = 0.801347229418  #feature_list = ['Sex', 'Age', 'Family_size', 'Cabin_cat', 'Title']

Cross-validated scores: 0.801347229418


/Users/ruyi/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [551]:
### KNeighbors clf
def Kneightbors_clf(features_train,labels_train, \
                    features_test,labels_test):

    from sklearn.neighbors import KNeighborsClassifier
    clf = KNeighborsClassifier(n_neighbors = 2)
    
    print 'clf name: KNeighbors'
    t0 = time.time()
    clf.fit(features_train, labels_train)
    print "training time:", round(time.time()-t0, 3), "s"

    t0 = time.time()
    pred = clf.predict(features_test)
    print "predicting time:", round(time.time()-t0, 3), "s"
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(pred, labels_test)
    print 'accuracy score:', accuracy

#Kneightbors_clf(train_features, train_labels, \
#              test_features, test_labels)    
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 2)

scores = cross_val_score(clf, features, labels, cv=5)
print 'Cross-validated scores:', scores.mean()


Cross-validated scores: 0.760947511284


In [552]:
### SVC clf

def SVC_clf(features_train,labels_train, \
                    features_test,labels_test):

    from sklearn import svm
    clf = svm.SVC(C=1.0, kernel='rbf')
    
    print 'clf name: SVC'
    t0 = time.time()
    clf.fit(features_train, labels_train)
    print "training time:", round(time.time()-t0, 3), "s"

    t0 = time.time()
    pred = clf.predict(features_test)
    print "predicting time:", round(time.time()-t0, 3), "s"

    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(pred, labels_test)
    print 'accuracy score:', accuracy

#SVC_clf(train_features, train_labels, \
#              test_features, test_labels)



from sklearn import svm
clf = svm.SVC(C=1.0, kernel='rbf')

scores = cross_val_score(clf, features, labels, cv=5)
print 'Cross-validated scores:', scores.mean()

clf.fit(features, labels)

Cross-validated scores: 0.789031405564


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [343]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
#from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing, svm
from sklearn.grid_search import GridSearchCV #sklearn version 0.15
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

scaler = preprocessing.MinMaxScaler()
skb = SelectKBest()
clf = svm.SVC()
pipe = Pipeline([('Scale_Features',scaler),('SKB', skb),('Classifier',clf)])

#print sorted(pipe.get_params().keys())

params = {'SKB__k':range(1,len(feature_list)),'Classifier__kernel':['linear', 'rbf'], 'Classifier__C':[1, 10, 20]}
my_clf = GridSearchCV(pipe, param_grid=params, scoring='f1_weighted')
my_clf.fit(train_features, train_labels)


pred = my_clf.predict(test_features)
print("Best estimator found by grid search:")
print my_clf.best_estimator_
print('Best Params found by grid search:')
print my_clf.best_params_

Best estimator found by grid search:
Pipeline(steps=[('Scale_Features', MinMaxScaler(copy=True, feature_range=(0, 1))), ('SKB', SelectKBest(k=1, score_func=<function f_classif at 0x1159b26e0>)), ('Classifier', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
Best Params found by grid search:
{'Classifier__kernel': 'linear', 'Classifier__C': 1, 'SKB__k': 1}


In [185]:
# Tune parameters of Naives Bayes with GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, \
                            recall_score, f1_score

skb = SelectKBest()
clf = GaussianNB()
pipe = Pipeline([('SKB', skb),('Classifier',clf)])


In [563]:
# Test dataset
test = pd.read_csv('test.csv')
print test.head()
featuring_process(test)
assign_into_object(test, feature_list)


unknownAges_test = test.loc[(test.Age.isnull())]
predictAges_test = regr.predict(unknownAges_test[age_feature_list].values)
test.loc[(test.Age.isnull()), 'Age'] = predictAges_test    


print test.isnull().sum()

    
'''
assign_into_object(test, feature_list)

print feature_list
'''
test_features = test[feature_list].values

pred = clf.predict(test_features)


   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


/Users/ruyi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Family_size      0
Cabin_cat        0
dtype: int64


In [564]:
import numpy as np

# Create a data frame with two columns: PassengerId & Survived. Survived contains your predictions
PassengerId =np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(pred, PassengerId, columns = ["Survived"])
print(my_solution)

# Check that your data frame has 418 entries
print(my_solution.shape)

# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_8.csv", index_label = ["PassengerId"])

      Survived
892          0
893          1
894          0
895          0
896          1
897          0
898          1
899          0
900          1
901          0
902          0
903          0
904          1
905          0
906          1
907          1
908          0
909          0
910          1
911          1
912          0
913          0
914          1
915          0
916          1
917          0
918          1
919          0
920          1
921          0
...        ...
1280         0
1281         0
1282         0
1283         1
1284         0
1285         0
1286         0
1287         1
1288         0
1289         1
1290         0
1291         0
1292         1
1293         0
1294         1
1295         0
1296         0
1297         0
1298         0
1299         0
1300         1
1301         1
1302         1
1303         1
1304         1
1305         0
1306         1
1307         0
1308         0
1309         0

[418 rows x 1 columns]
(418, 1)


In [566]:
s2 = pd.read_csv("my_solution_2.csv")
s3 = pd.read_csv("my_solution_3.csv")
s4 = pd.read_csv("my_solution_4.csv")
s5 = pd.read_csv("my_solution_5.csv")
s6 = pd.read_csv("my_solution_6.csv")
s7 = pd.read_csv("my_solution_7.csv")
s8 = pd.read_csv("my_solution_8.csv")




In [567]:
s7.equals(s8)

False

In [568]:
def count_survival(df):
    print df['Survived'].sum()

count_survival(s2)
count_survival(s3)    
count_survival(s4)
count_survival(s5)
count_survival(s6)
count_survival(s7)
count_survival(s8)

175
152
184
139
152
149
151
